In [1]:
import tensorflow as tf
import os
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

if not os.path.isdir('models'):
    os.mkdir('models')
    
print('TensorFlow version:', tf.__version__)
print('Is using GPU?', tf.test.is_gpu_available())

TensorFlow version: 2.11.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is using GPU? False


In [2]:
import cv2
import keras
from keras.models import Sequential
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv('Data/train_new.csv')
train.head()

,image,class
0,harassment_shv1.mp4_frame0.jpg,1
1,harassment_shv1.mp4_frame1.jpg,1
2,harassment_shv1.mp4_frame10.jpg,1
3,harassment_shv1.mp4_frame11.jpg,1
4,harassment_shv1.mp4_frame12.jpg,1


In [4]:
import numpy as np
from tqdm import tqdm
from PIL import Image
from tensorflow.keras.preprocessing import image

# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = Image.open('train/' + train['image'][i]).resize((224, 224))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img / 255.0
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|██████████| 757/757 [00:22<00:00, 34.11it/s]


(757, 224, 224, 3)

In [40]:
y = train['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=80, test_size=0.25)

In [41]:
y_train.value_counts()

0    353
1    214
Name: class, dtype: int64

In [7]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dropout, Flatten, Input, Dense
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.models import Model

def create_model():
    
    def add_conv_block(model, num_filters):
        
        model = Conv2D(num_filters, 3, activation='relu', padding='same')(model)
        model = BatchNormalization()(model)
        model = SeparableConv2D(num_filters, 3, activation='relu', padding='valid')(model)
        model = MaxPooling2D(pool_size=2)(model)
        model = Dropout(0.2)(model)

        return model
    
    inputs = Input(shape=(224, 224, 3))
    
    x = add_conv_block(inputs, 32)
    x = add_conv_block(x, 64)
    x = add_conv_block(x, 128)
    x = add_conv_block(x, 256)
    
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(2, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 224, 224, 32)     128       
 ormalization)                                                   
                                                                 
 separable_conv2d (Separable  (None, 222, 222, 32)     1344      
 Conv2D)                                                         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                             

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the number of epochs and batch size
epochs = 10
batch_size = 128

# Set up a checkpoint to save the best model weights
checkpoint = ModelCheckpoint('model.h5', save_best_only=True)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[checkpoint]
)

Epoch 1/10
5/5 [==============================] - 384s 77s/step - loss: 1.1093 - accuracy: 0.5421 - val_loss: 0.6916 - val_accuracy: 0.6250
Epoch 2/10
5/5 [==============================] - 408s 81s/step - loss: 0.7287 - accuracy: 0.5868 - val_loss: 0.6923 - val_accuracy: 0.6250
Epoch 3/10
5/5 [==============================] - 407s 81s/step - loss: 0.6484 - accuracy: 0.6281 - val_loss: 0.6911 - val_accuracy: 0.6250
Epoch 4/10
5/5 [==============================] - 387s 77s/step - loss: 0.6303 - accuracy: 0.6529 - val_loss: 0.6897 - val_accuracy: 0.6250
Epoch 5/10
5/5 [==============================] - 398s 80s/step - loss: 0.5899 - accuracy: 0.6711 - val_loss: 0.6890 - val_accuracy: 0.6250
Epoch 6/10
5/5 [==============================] - 393s 78s/step - loss: 0.5530 - accuracy: 0.7058 - val_loss: 0.6891 - val_accuracy: 0.6250
Epoch 7/10
5/5 [==============================] - 401s 80s/step - loss: 0.4875 - accuracy: 0.7587 - val_loss: 0.6887 - val_accuracy: 0.6250
Epoch 8/10
5/5 [====

In [9]:
# getting the test list
f = open("test.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating the dataframe
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test_videos = test['video_name']
test.shape

(9, 1)

In [10]:
# creating the tags
train = pd.read_csv('Data/train_new.csv')
y = train['class']
y = pd.get_dummies(y)

In [16]:
test_videos

0     harassment/harassment_shv4.mp4
1     harassment/harassment_shv8.mp4
2    harassment/harassment_shv14.mp4
3    harassment/harassment_shv18.mp4
4            non_harassment/non4.mp4
5            non_harassment/non8.mp4
6           non_harassment/non14.mp4
7           non_harassment/non18.mp4
8                                   
Name: video_name, dtype: object

In [32]:
from glob import glob
from scipy import stats as s
import os

# creating two lists to store predicted and actual tags
predict = []
actual = []

# Enable eager execution
tf.config.run_functions_eagerly(True)

# removing all files from the temp folder
files = glob('temp/*')
for f in files:
    os.remove(f)

count = 0
# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    
    videoFile = test_videos[i]
    videoPath = os.path.join('Data', videoFile)
    cap = cv2.VideoCapture(videoPath)  # capturing the video from the given path
    
    if not cap.isOpened():
        print("halo")
        # Skip iteration if video capture fails
        continue
    
    frameRate = cap.get(5)  # frame rate
    
    while cap.isOpened():
        frameId = cap.get(1)  # current frame number
        ret, frame = cap.read()
        
        if not ret:
            break
        
        if frameId % 3 == 0:
            # storing the frames of this particular video in temp folder
            filename = os.path.join('temp', f'_frame{count}.jpg')
            cv2.imwrite(filename, frame)
            count += 1
    
    cap.release()
    
    # reading all the frames from temp folder
    images = glob("temp/*.jpg")
    
    if len(images) == 0:
        # Skip iteration if no valid images found
        continue
    
    prediction_images = []
    for img_path in images:
        img = image.load_img(img_path, target_size=(224, 224, 3))
        img = image.img_to_array(img)
        img = img / 255.0
        prediction_images.append(img)
        
    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    
    # predicting tags for each array
    prediction = model.predict_on_batch(prediction_images)
    
    # converting features to one-dimensional array
    prediction_labels = np.argmax(prediction, axis=1)
    print(prediction_labels)
    
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(1 if np.sum(prediction_labels) >= 5 else 0)
    
    # appending the actual tag of the video
    if videoFile.split('/')[1].split('_')[0] == 'harassment':
        actual.append(1)
    else:
        actual.append(0)

 11%|█         | 1/9 [00:02<00:17,  2.21s/it]

[0 0 0 0 0 0 0 0 0 0 0]


 22%|██▏       | 2/9 [00:05<00:20,  2.88s/it]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


 33%|███▎      | 3/9 [00:09<00:21,  3.58s/it]

...

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0]


 89%|████████▉ | 8/9 [01:25<00:17, 17.01s/it]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0]


100%|██████████| 9/9 [01:26<00:00,  9.61s/it]

halo


In [58]:
images_test = glob("frame_testing/*.jpg")
print(images_test)
testing_predict = []
for img_path in images_test:
        img = image.load_img(img_path, target_size=(224, 224, 3))
        img = image.img_to_array(img)
        img = img / 255.0
        img = np.expand_dims(img, axis=0)
        # predicting tags for each array
        testing_prediction = model.predict(img)

        # converting features to one-dimensional array
        pred = np.argmax(testing_prediction, axis=1)
        print(pred)
        testing_predict.append(pred)
print(testing_predict)

['frame_testing\\_frame0.jpg', 'frame_testing\\_frame1.jpg', 'frame_testing\\_frame2.jpg', 'frame_testing\\_frame3.jpg', 'frame_testing\\_frame4.jpg', 'frame_testing\\_frame5.jpg', 'frame_testing\\_frame6.jpg', 'frame_testing\\_frame7.jpg', 'frame_testing\\_frame8.jpg', 'frame_testing\\_frame9.jpg']
1/1 [==============================] - 0s 130ms/step
[0]
1/1 [==============================] - 0s 117ms/step
[0]
1/1 [==============================] - 0s 113ms/step
[0]
1/1 [==============================] - 0s 130ms/step
[0]
1/1 [==============================] - 0s 116ms/step
[0]
1/1 [==============================] - 0s 132ms/step
[0]
1/1 [==============================] - 0s 119ms/step
[0]
1/1 [==============================] - 0s 118ms/step
[0]
1/1 [==============================] - 0s 114ms/step
[0]
1/1 [==============================] - 0s 116ms/step
[0]
[array([0], dtype=int64), array([0], dtype=int64), array([0], dtype=int64), array([0], dtype=int64), array([0], dtype=int64), arr

In [13]:
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

50.0

In [14]:
print(predict)

[0, 0, 0, 0, 0, 0, 0, 0]


In [15]:
print(actual)

[1, 1, 1, 1, 0, 0, 0, 0]
